In [2]:
import os
import sys 
module_path = '/mnt/nvme1n1/Aamir/CurePilot/'
if module_path not in sys.path:
    sys.path.append(module_path)

# from git import Repo
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain_community.document_loaders.generic import GenericLoader
# from langchain_community.document_loaders.parsers import LanguageParser
from language.language_parser import LanguageParser
from langchain_community.vectorstores import Milvus

In [3]:
repo_path = "/mnt/nvme1n1/Aamir/CurePilot/CodeGPT/"

In [6]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=1000),
)
documents = loader.load()
len(documents)

2

In [7]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=500, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

2

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding_model = '/mnt/sda/e5-mistral-7b-instruct/'

db = Milvus.from_documents(texts, HuggingFaceEmbeddings(model_name=embedding_model), 
                           collection_name="CodeGPT", connection_args={"host": "127.0.0.1", "port": "19530"})
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [7]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline

In [8]:
#################################################################
# Tokenizer
#################################################################

model_name = '/mnt/sda/Mistral_v0.2/'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards: 100%|██████████| 3/3 [02:17<00:00, 45.77s/it]


In [9]:
response_generation_pipeline = pipeline(
 model=mistral_model,
 tokenizer=tokenizer,
 task="text-generation",
 temperature=0.2,
 repetition_penalty=1.1,
 return_full_text=True,
 max_new_tokens=1000,
)
response_generation_llm = HuggingFacePipeline(pipeline=response_generation_pipeline)

In [10]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model_name="gpt-3.5-turbo",
#     api_key="",
# )
memory = ConversationSummaryMemory(
    llm=response_generation_llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(response_generation_llm, retriever=retriever, memory=memory)

In [11]:
question = "act as an expert pair programmer. you have been placed to work on a large project. embeddings have been generated \
    for the whole code base and are placed in vector db. you'll be tasked to either write code or answer few questions based \
        on the project codebase. moreover, relevant documents from the vectordb will be passed to you. project is in csharp hence, please only write code in csharp. make sure your code works with the existing code. Question: what are the available members of PredictionDrugDto. also list their visibility"
result = qa(question)
result["answer"]

/home/aamira171/.local/share/virtualenvs/CurePilot-Mzm1XPrs/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/aamira171/.local/share/virtualenvs/CurePilot-Mzm1XPrs/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aamira171/.local/share/virtualenvs/CurePilot-Mzm1XPrs/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You sh

' The PredictionDrugDto class has the following public properties and their respective types:\n\n```csharp\npublic int DrugId { get; set; } // int\npublic string Name { get; set; } // string\npublic double PredictedDose { get; set; } // double\npublic DateTime StartTime { get; set; } // DateTime\npublic TimeSpan Duration { get; set; } // TimeSpan\n```'